In [18]:
import tensorflow as tf
from sklearn.datasets import load_digits
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelBinarizer

from keras import backend as K
K.clear_session()          # 清除之前会话，防止jupyter缓存影响

# 加载数据
digits = load_digits()
X = digits.data      # 0-9图片数据
y = digits.target    # 图片对应标签
y = LabelBinarizer().fit_transform(y)    # 标签，长度为10，在数据对应位置标1，其他位置为0
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)   # 测试数据比例为30%

def add_layer(inputs, in_size, out_size, layer_name, activation_function=None):
    '''添加网络层'''
    Weights = tf.Variable(tf.random_normal([in_size, out_size]))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    tf.summary.histogram(layer_name + '/outputs', outputs)
    return outputs

# 传入值
xs = tf.placeholder(tf.float32, [None, 64])  # 8×8
ys = tf.placeholder(tf.float32, [None, 10])

# 添加层
l1 = add_layer(xs, 64, 100, 'l1', activation_function=tf.nn.tanh)   # 输出100是为了测试过拟合问题
prediction = add_layer(l1, 100, 10, 'l2', activation_function=tf.nn.softmax)

# 损失，训练优化器
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(prediction), reduction_indices=[1]))
tf.summary.scalar('loss', cross_entropy)
train_step = tf.train.GradientDescentOptimizer(0.6).minimize(cross_entropy)

sess = tf.Session()
merged = tf.summary.merge_all()

# 输出两个图信息
train_writer = tf.summary.FileWriter('013logs/train', sess.graph)
test_writer = tf.summary.FileWriter('013logs/test', sess.graph)

sess.run(tf.initialize_all_variables())

for i in range(500):
    # print(i)
    sess.run(train_step, feed_dict={xs:X_train, ys:y_train})
    if i%50 == 0:
        train_result = sess.run(merged, feed_dict={xs:X_train, ys:y_train})
        test_result = sess.run(merged, feed_dict={xs:X_test, ys:y_test})
        train_writer.add_summary(train_result, i)
        test_writer.add_summary(test_result, i)

### 使用dropuout处理过拟合

In [4]:
import tensorflow as tf
from sklearn.datasets import load_digits
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelBinarizer

from keras import backend as K
K.clear_session()          # 清除之前会话，防止jupyter缓存影响

# 加载数据
digits = load_digits()
X = digits.data      # 0-9图片数据
y = digits.target    # 图片对应标签
y = LabelBinarizer().fit_transform(y)    # 标签，长度为10，在数据对应位置标1，其他位置为0
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)   # 测试数据比例为30%

def add_layer(inputs, in_size, out_size, layer_name, activation_function=None):
    '''添加网络层'''
    Weights = tf.Variable(tf.random_normal([in_size, out_size]))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    Wx_plus_b = tf.nn.dropout(Wx_plus_b, keep_prob)    # 添加dropout功能
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    tf.summary.histogram(layer_name + '/outputs', outputs)
    return outputs

# 传入值
keep_prob = tf.placeholder(tf.float32)        # 保持多少的比例不被 drop 掉 
xs = tf.placeholder(tf.float32, [None, 64])  # 8×8
ys = tf.placeholder(tf.float32, [None, 10])

# 添加层
l1 = add_layer(xs, 64, 50, 'l1', activation_function=tf.nn.tanh)   # 输出100是为了测试过拟合问题
prediction = add_layer(l1, 50, 10, 'l2', activation_function=tf.nn.softmax)

# 损失，训练优化器
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(prediction), reduction_indices=[1]))
tf.summary.scalar('loss', cross_entropy)
train_step = tf.train.GradientDescentOptimizer(0.6).minimize(cross_entropy)

sess = tf.Session()
merged = tf.summary.merge_all()

# 输出两个图信息
train_writer = tf.summary.FileWriter('013logs/train', sess.graph)
test_writer = tf.summary.FileWriter('013logs/test', sess.graph)

sess.run(tf.initialize_all_variables())

for i in range(500):
    # 添加dropout比例，分别设置为 1 和 0.5 使用tensorboard查看损失
    sess.run(train_step, feed_dict={xs:X_train, ys:y_train, keep_prob:0.5})  # 保持50%不被 drop 掉
    if i%50 == 0:
        train_result = sess.run(merged, feed_dict={xs:X_train, ys:y_train, keep_prob:1})  # 记录时保持所有数据，不drop
        test_result = sess.run(merged, feed_dict={xs:X_test, ys:y_test, keep_prob:1})
        train_writer.add_summary(train_result, i)
        test_writer.add_summary(test_result, i)

##### cmd窗口下：

> E:
> cd E:\wk\GithubProjects\DeepLearning\莫烦教程
> tensorboard --logdir=013logs/

##### 浏览器打开链接：

> http://win-q2dq03poefr:6006/#scalars

##### test与train的loss（keep_prob=1）：

![keep_prob=1](keep_prob=1.PNG)

##### test与train的loss（keep_prob=0.5）：

![keep_prob=0.5](keep_prob=0.5.PNG)